In [92]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os.path as osp
import pandas as pd
from PIL import Image
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import torchvision
from torchvision import models, transforms
from tqdm import tqdm

%matplotlib inline

In [40]:
def set_seed(seed=1234):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(1234)

In [42]:
def set_gpu(gpu='cude:0'):
    device = torch.device(gpu if torch.cuda.is_available() else 'cpu')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    return device

device = set_gpu('cuda:0')

In [53]:
train_label_master = pd.read_csv('./data/train_master.tsv', sep='\t')
label_master = pd.read_csv('./data/label_master.tsv', sep='\t')
train_label_master = train_label_master.merge(label_master, how='left', on='label_id')

In [95]:
class ImageTransform():
    """
    Image preprocessing class.
    Make transforms for train and val as a dictionary.
    Data augmentation is performed for training.

    Attributes
    ----------
    resize: int
        Image size after resized
    mean: (R, G, B)
        Mean value of each channel
    std: (R, G, B)
        Std value of each channel
    """
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase: 'train' or 'val' or 'test'
            Specify the preprocessing mode.
        """
        return self.data_transform[phase](img)

In [101]:
def make_data_path_list(phase='train', rootpath='./data/'):

    target_path = osp.join(rootpath+phase+'/**.png')
    path_list = []

    for path in glob.glob(target_path):
        path_list.append(path)
        
    return path_list

In [109]:
class ImageDataset(Dataset):
    def __init__(self, file_list, train_label_master, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):

        img_path = self.file_list[index]
        img = Image.open(img_path)

        img_transformed = self.transform(
            img, self.phase)

        label = train_label_master[train_label_master['file_name']==img_path[13:]]['label_id'].item()

        return img_transformed, label

In [119]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('---------------')

    for phase in ['train', 'val']:
        if phase == 'train':
            net.train()
        else:
            net.eval()

        epoch_loss = 0.0
        epoch_corrects = 0

        if (epoch == 0) and (phase == 'train'):
            continue

        for inputs, labels in tqdm(dataloaders_dict[phase]):
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase=='train'):
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            epoch_loss += loss.item() * inputs.size(0)

            epoch_corrects += torch.sum(preds == labels.data)

        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double(
        ) / len(dataloders_dict[phase].dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

In [111]:
size = 32
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

train_val_list = make_data_path_list(phase='train', rootpath='./data/')
random.shuffle(train_val_list)
train_list = train_val_list[:int(len(train_val_list)*0.8)]
val_list = train_val_list[int(len(train_val_list)*0.8):]

train_dataset = ImageDataset(file_list=train_list, train_label_master=train_label_master, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = ImageDataset(file_list=val_list, train_label_master=train_label_master, transform=ImageTransform(size, mean, std), phase='val')

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

In [112]:
net = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
net.classifier[6] = nn.Linear(in_features=4096, out_features=20)
net.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=params_to_update, lr=0.001)

params_to_update = []
update_param_names = ['classifier.6.weight', 'classifier.6.bias']

for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.required_grad = False

classifier.6.weight
classifier.6.bias


In [ ]:
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/2
---------------
Epoch 2/2
---------------


 78% 976/1250 [22:08<06:25,  1.41s/it]